In [32]:
import glob                         # this module helps in selecting files
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

In [33]:
import pandas as pd
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2023-06-03 18:17:28--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4,1K) [application/zip]
Saving to: 'datasource.zip.1'

     0K ....                                                  100% 1,58G=0s

2023-06-03 18:17:29 (1,58 GB/s) - 'datasource.zip.1' saved [4249/4249]



In [34]:
import zipfile

with zipfile.ZipFile("datasource.zip", "r") as extracted_file:
    extracted_file.extractall("dealership_data")

In [35]:
temp_file = "dealership_temp.tmp"
log_file = "dealership_log_file.txt"
target_file = "dealership_transofrmed_data_file"

In [36]:
def extract_csv(file_name):
    dataframe = pd.read_csv(file_name)
    return dataframe

def extract_json(file_name):
    dataframe = pd.read_json(file_name, lines=True)
    return dataframe

def extract_xml(file_name):

    dataframe = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price", "fuel"])
    tree = ET.parse(file_name)
    root = tree.getroot()

    for car in root:
        car_model = car.find("car_model").text
        year_of_manufacture = int(car.find("year_of_manufacture").text)
        price = float(car.find("price").text)
        fuel = car.find("fuel").text
        dataframe = pd.concat([dataframe, pd.DataFrame({"car_model": [car_model], "year_of_manufacture": [year_of_manufacture], "price": [price], "fuel": [fuel]})])

    return dataframe

In [37]:
def extract():
    extracted_data = pd.DataFrame(columns=["car_model","year_of_manufacture","price","fuel"])

    for csv_file in glob.glob("dealership_data/*.csv"):
        extracted_data = pd.concat([extracted_data, extract_csv(csv_file)], ignore_index= True)

    for json_file in glob.glob("dealership_data/*.json"):
        extracted_data = pd.concat([extracted_data, extract_json(json_file)], ignore_index= True)

    for xml_file in glob.glob("dealership_data/*.xml"):
        extracted_data = pd.concat([extracted_data, extract_xml(xml_file)], ignore_index= True)

    return extracted_data

In [38]:
def transform(data):
    data["price"] = round(data.price, 2)
    return data

In [39]:
def load(target_file, data_load):
    data_load.to_csv(target_file)

In [40]:
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'
    now = datetime.now()
    time_stamp = now.strftime(timestamp_format)
    with open(log_file, "w") as logfile:
        logfile.write(time_stamp + ',' + message + '\n')


In [41]:
log("Extract phase started")
extracted_data = extract()
log("Extract phase ended")
extracted_data

log("Transoform phase started")
transformed_data = transform(extracted_data)
log("Transoform phase ended")

log("Load phase started")
load(target_file, transformed_data)
log("Load phase ended")
